Textcleaner: das mittlere nehmen
nur 1000-2000 Zeilen
2 Splitter, 2 Similarity
anderes Modell nehmen zum evaluieren (1 Variante) -> 4 müssen noch rausgenommen werden

1. Question
2. Answer to Evaluate
3. Context to Evaluate
4. Ground Truth

## Evaluierung Embedding

- mit samples arbeiten von question, answer, contexts, ground truth
- score berechnen (Faithfulness ist eine Formel -> hoch, dann ist der Kontext zum Groundtruth hoch)

In [1]:
from credentials import AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT
from langchain.embeddings import AzureOpenAIEmbeddings
import chromadb
from langchain_community.vectorstores import Chroma
from pathlib import Path

# Embedding-Funktion (muss identisch sein wie beim Erstellen der DB!)
embedding_model = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-05-15",
    deployment="text-embedding-3-large"  # Muss mit dem Deployment-Namen übereinstimmen
)

# Pfad zur zu ladenden DB
persist_directory = Path.cwd().parent / "src" / "chroma_recchar_800_cosine"

collection_name = "recchar_800_cosine"

# Client und Collection prüfen
client = chromadb.PersistentClient(path=str(persist_directory))
collection_names = client.list_collections()

if collection_name not in collection_names:
    raise ValueError(f"Collection '{collection_name}' nicht gefunden in {persist_directory}.")

print(f"Collection '{collection_name}' gefunden. Laden...")

# Vector Store laden
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=collection_name,
    persist_directory=str(persist_directory)
)

# Dokument-Anzahl prüfen
doc_count = vector_store._collection.count()
print(f"Datenbank geladen: {doc_count} Dokumente gefunden.")

/var/folders/0l/jk3grg717z34yf9xh85kj0bh0000gn/T/ipykernel_33294/3406113087.py:8: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(


KeyError: 'chunk_size'

In [1]:
from evaluation import RAGAS_Evaluator

evaluator = RAGAS_Evaluator()

results, ragas_metrics, ir_metrics = evaluator.run(
    collection_name="chroma_recchar_800_cosine",
    retriever_name="similarity_k5",
    prompt_name="simple",
    temperature=0.1,
    save_path="eval_output.csv"
)

print("RAGAS:", ragas_metrics)
print("IR:", ir_metrics)
print("Durchschnittliche Antwortähnlichkeit:", results["similarity_to_gold"].mean())


/Users/husazwerg/code/gitlab.fhnw.ch/gruppen_arbeit/npr_fs25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-30 10:17:54,503 - INFO - Lade Testdaten aus Parquet-Datei ...
2025-04-30 10:17:54,760 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


ValueError: Collection 'chroma_recchar_800_cosine' nicht gefunden.

In [ ]:

def plot_eval_result(self, df: pd.DataFrame):

    sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

    columns = [
        "faithfulness",
        "answer_relevancy",
        "context_precision",
        "context_entity_recall",
        "answer_similarity",
        "answer_correctness",
    ]

    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df[columns], palette="Set1", width=0.6, linewidth=1.5)
    plt.title(f"{self.name}: Ragas Metrics boxplot", fontsize=16)
    plt.ylabel("Score", fontsize=14)
    plt.xticks(fontsize=12, rotation=20)
    plt.tight_layout()
    plt.show()

def plot_eval_result_bar(self, df: pd.DataFrame):
    """
    Plot a barplot of evaluation scores for RAGAS metrics, MRR, precision@2, and recall@2.
    Args:
        results (pd.DataFrame): The results DataFrame from the evaluation.
    """
    sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

    ragas_metrics = [
        "faithfulness",
        "answer_relevancy",
        "context_precision",
        "context_entity_recall",
        "semantic_similarity",
        "answer_correctness",
    ]
    additional_metrics = ["MRR", "precision@2", "recall@2"]
    all_metrics = ragas_metrics + additional_metrics

    metric_means = df[all_metrics].mean()

    plt.figure(figsize=(12, 6))
    sns.barplot(
        x=metric_means.index,
        y=metric_means.values,
        palette="Set1",
        hue=metric_means.index,
        legend=False,
    )
    plt.title(f"{self.name}: Ragas + Non-LLM Metrics (Mean)", fontsize=16)
    plt.ylabel("Mean Score", fontsize=14)
    plt.xlabel("Metrics", fontsize=14)
    plt.xticks(fontsize=12, rotation=20)
    plt.tight_layout()
    plt.show()

def plot_results_all(self, df: pd.DataFrame):
    """
    Display both the boxplot for RAGAS metrics and the barplot for RAGAS + non-LLM metrics.
    Args:
        df (pd.DataFrame): DataFrame containing raw RAGAS metric values for the boxplot.
        results (pd.DataFrame): DataFrame containing RAGAS and non-LLM metric scores for the barplot.
    """
    sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

    # Boxplot: RAGAS Metrics
    boxplot_columns = [
        "faithfulness",
        "answer_relevancy",
        "context_precision",
        "context_entity_recall",
        "semantic_similarity",
        "answer_correctness",
    ]

    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df[boxplot_columns], palette="Set1", width=0.6, linewidth=1.5)
    plt.title(f"{self.name}: RAGAS Metrics Boxplot", fontsize=16)
    plt.ylabel("Score", fontsize=14)
    plt.xticks(fontsize=12, rotation=20)
    plt.tight_layout()
    plt.show()

    # Barplot: RAGAS + Non-LLM Metrics
    ragas_metrics = [
        "faithfulness",
        "answer_relevancy",
        "context_precision",
        "context_entity_recall",
        "semantic_similarity",
        "answer_correctness",
    ]
    additional_metrics = ["MRR", "precision@2", "recall@2"]
    all_metrics = ragas_metrics + additional_metrics

    metric_means = df[all_metrics].mean()

    plt.figure(figsize=(12, 6))
    sns.barplot(
        x=metric_means.index,
        y=metric_means.values,
        palette="Set1",
        hue=metric_means.index,
        legend=False,
    )
    plt.title(f"{self.name}: RAGAS + Non-LLM Metrics (Mean)", fontsize=16)
    plt.ylabel("Mean Score", fontsize=14)
    plt.xlabel("Metrics", fontsize=14)
    plt.xticks(fontsize=12, rotation=20)
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import langdetect
import re

from collections import Counter
from collections import OrderedDict
from langdetect import detect

import torch

import sys
import os
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath("../"))
import src.utilities as u
import src.plots as p
import src.constants as c
c_stopwords = c.custom_stopwords

from src.utilities import TextCleaner
from src.credentials import AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings


from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import chromadb

#!pip install langchain-community
#!pip install openai
#!pip install tiktoken

df_train_1000 = pd.read_parquet('../data_mc1/data_processed/df_train_1000.parquet')
df_train = pd.read_parquet('../data_mc1/data_processed/df_train_cleaned.parquet')
df_eval = pd.read_parquet('../data_mc1/data_processed/cleantech_rag_evaluation_data.parquet')

In [ ]:
azure_embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-large", # gleich wie bei Embedden der Chunks
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
)


openai_llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    temperature=0.0 
)

# Schritt 1: Bereite das Dataset vor
df_ragas_input = df_test_embedd[[
    "question", 
    "relevant_text_llm",    
    "answer_llm", 
    "answer",
    "relevant_text"
]].copy()

# Spalten umbenennen
df_ragas_input = df_ragas_input.rename(columns={
    "relevant_text_llm": "retrieved_contexts",
    "relevant_text": "reference"
})

# Typen konvertieren
for col in ["question", "answer_llm", "answer", "reference"]:
    df_ragas_input[col] = df_ragas_input[col].astype(str)

# Schritt 2: Hugging Face Dataset
ragas_dataset = Dataset.from_pandas(df_ragas_input)

# Schritt 3: Evaluation starten
metrics = [faithfulness, answer_relevancy, context_precision]

ragas_result = evaluate(
    ragas_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision],
    embeddings=azure_embeddings,
    llm=openai_llm
)

# Schritt 4: Ausgabe
print("RAGAS Evaluation (mit GPT-4o-mini bewertet):")
print(ragas_result)

# Schritt 5: Optional – Vergleich mit Goldstandard
def simple_similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

df_ragas_input["similarity_to_gold"] = df_ragas_input.apply(
    lambda row: simple_similarity(row["answer_llm"], row["answer"]), axis=1
)

mean_similarity = df_ragas_input["similarity_to_gold"].mean()

print("\nVergleich answer_llm vs. Goldstandard:")
print(f"Durchschnittliche Ähnlichkeit (answer_llm vs. answer): {mean_similarity:.3f}")
